# Lecture 8(a)

## Public, Protected, Private

In most objected oriented languages, data encapsulation is achieved by enabling classes to declair protected and private data members in addition to the public ones. Quick recap:

* Public members are accessible to everyone
* Private members are only accessible to the class itself
* Protected members are accessiblve to the all instances of the same class (including child classes) 

Python's implementation of data encapsulation is not strictly enforced by the langauge and is mostly a convention. Data members starting with one underscore (`_`) are protected. Data members starting with two underscores (`__`) are private.

Consider the following example:

In [1]:
class parent:
    def __init__(self):
        self.public="I'm Public"
        self._protected="I'm Protected"
        self.__private="I'm Private"
        
    def set_private_parent(self,v):
        self.__private=v
        
    def get_private_parent(self):
        return self.__private
        
class child(parent):
    def __init__(self, init_parent=True):
        if init_parent:
            super(child,self).__init__()
    
    def set_public(self,v):
        self.public=v
        
    def set_protected(self,v):
        self._protected=v

    def set_private(self,v):
        self.__private=v
        
    def get_public(self):
        return self.public
        
    def get_protected(self):
        return self._protected

    def get_private(self):
        return self.__private


First note that because we declared the data members in the parent constructor, we have to make sure that the child calls the parent constructor.

In [2]:
child_instance = child(init_parent=False)
child_instance.public

AttributeError: 'child' object has no attribute 'public'

Even though we wrote accessors (setters and getters), we can directly access and set the data:

In [3]:
child_instance = child()
print(child_instance.public)
child_instance.public="Changed Public"
print(child_instance.public)

I'm Public
Changed Public


Of couse the accessors also work:

In [4]:
child_instance = child()
print(child_instance.get_public())
child_instance.set_public("Changed Public")
print(child_instance.get_public())

I'm Public
Changed Public


How about the protected?

In [5]:
print(child_instance._protected)
child_instance._protected="Changed Protected"
print(child_instance._protected)

I'm Protected
Changed Protected


In [6]:
child_instance = child()
print(child_instance.get_protected())
child_instance.set_protected("Changed Protected")
print(child_instance.get_protected())

I'm Protected
Changed Protected


So there isn't any difference between public and protected in python. We can just adopt the convention that data members starting with a single underscore will be only accessed via accessors. 

How about private?

In [7]:
print(child_instance.__private)
child_instance.__private="Changed Private"
print(child_instance.__private)

AttributeError: 'child' object has no attribute '__private'

It appears that we finally have some protection. A closer look shows how its done:

In [8]:
dir(child_instance)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parent__private',
 '_protected',
 'get_private',
 'get_private_parent',
 'get_protected',
 'get_public',
 'public',
 'set_private',
 'set_private_parent',
 'set_protected',
 'set_public']

Note that instead of a data member `__private` we have a data member `_parent__private`. All python does is to replace anything that has the pattern `<class_name>.__<data_name>` to `<class_name>._<class_name>__<data_name>`. So in fact, we can still change this data member and there is no real protection:

In [9]:
print(child_instance._parent__private)
child_instance._parent__private="Changed Private"
print(child_instance._parent__private)

I'm Private
Changed Private


Just to make sure we understand, here what the parent class looks like:

In [10]:
parent_instance=parent()
dir(parent_instance)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parent__private',
 '_protected',
 'get_private_parent',
 'public',
 'set_private_parent']

Note that a child cannot change a parent's private data:

In [11]:
child_instance = child()
print(child_instance.get_private())
child_instance.set_private("Changed Private")
print(child_instance.get_private())

AttributeError: 'child' object has no attribute '_child__private'

In [12]:
child_instance = child()
print(child_instance.get_private_parent())
child_instance.set_private_parent("Changed Private")
print(child_instance.get_private_parent())

I'm Private
Changed Private


Why don't I get an error in the following case? 

In [13]:
child_instance = child()
print(child_instance.get_private_parent())
child_instance.set_private("Changed Private")
print(child_instance.get_private_parent())

I'm Private
I'm Private


Note that the code doesn't work as intended... why? 

See if you can figure it out by looking at:

In [14]:
dir(child_instance)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_child__private',
 '_parent__private',
 '_protected',
 'get_private',
 'get_private_parent',
 'get_protected',
 'get_public',
 'public',
 'set_private',
 'set_private_parent',
 'set_protected',
 'set_public']

## Data Serialization

Data serialization refers to the process of converting data (usually in memory) that may have complex structure (e.g. a tree), into a linear sequence that can be use to reconstitute the original data structure. Such a sequence can be stored in a file or transmitted over a network. 

For example consider the following "simple" data structure:

In [15]:
# Simple Data Type

data_dict = { "A": 1, 
              "B": "Foo"}

### Python `repr`

The python `repr` method of build-ins and classes you implement can be used as a means of serialization. Take any python built in and you can see it's string representation, which is essentially a string of python code that can evaluates to the object:

In [16]:
repr(data_dict)

"{'A': 1, 'B': 'Foo'}"

This representation can be easily written to a file:

In [17]:
with open('file.py',"w") as f: 
    f.write(repr(data_dict))

In [18]:
!cat file.py

{'A': 1, 'B': 'Foo'}

And reconstituted by evaluating the contents of the file:

In [19]:
with open('file.py', 'r') as f: 
    data_dict_reloaded = eval(f.read())

data_dict_reloaded

{'A': 1, 'B': 'Foo'}

Note that `eval` uses the python interpreter to execute python expressions stored in strings:

In [20]:
eval("print('Hello World')")

Hello World


In [21]:
x=eval("1+1")
x

2

### YAML

There are other standard formats for storing simple data types. For example YAML:

In [23]:
import yaml
yaml.dump(data_dict)

'A: 1\nB: Foo\n'

In [24]:
with open('file.yaml',"w") as f: 
    f.write(yaml.dump(data_dict))

In [25]:
!cat file.yaml

A: 1
B: Foo


In [26]:
!ls 

Lecture.8.a.ipynb Quiz.2.ipynb      file.py
Lecture.8.b.ipynb Scores.csv        file.yaml


In [27]:
with open('file.yaml', 'r') as f: 
    data_dict_reloaded = yaml.safe_load(f.read())

data_dict_reloaded

{'A': 1, 'B': 'Foo'}

### JSON

[JSON](https://www.json.org/json-en.html) is commonly used to transmit data on the web:

In [28]:
import json
json.dumps(data_dict)

'{"A": 1, "B": "Foo"}'

In [29]:
with open('file.json',"w") as f: 
    json.dump(data_dict,f)

In [30]:
!cat file.json

{"A": 1, "B": "Foo"}

In [31]:
with open('file.json', 'r') as f: 
    data_dict_reloaded = json.load(f)

data_dict_reloaded

{'A': 1, 'B': 'Foo'}

### XML

XML is another format commonly used for storing data. It allows a bit more structure and there are python tools for creating XML representations of data, but it's a bit more complicated than the example above, so we'll skip it for now.

### pickle

[pickle](https://docs.python.org/3/library/pickle.html) is python's method of serialing objects. Some advantages are that it is a binary format, so it is more compact, and that it can store full python objects, not just simple built-ins. Lets look at the [pickle documentation](https://docs.python.org/3/library/pickle.html) first.

Here is an example:

In [32]:
import pickle
pickle.dumps(data_dict,protocol=2)

b'\x80\x02}q\x00(X\x01\x00\x00\x00Aq\x01K\x01X\x01\x00\x00\x00Bq\x02X\x03\x00\x00\x00Fooq\x03u.'

In [33]:
with open('file.pickle',"wb") as f: 
    pickle.dump(data_dict,f)

In [34]:
!cat file.pickle

��       }�(�A�K�B��Foo�u.

In [35]:
with open('file.pickle', 'rb') as f: 
    data_dict_reloaded = pickle.load(f)

data_dict_reloaded

{'A': 1, 'B': 'Foo'}

## Python classes

Imagine you have data stored in a python object:

In [36]:
# Instance of a python class with data

class data_class:
    def __init__(self):
        self._data = dict()
    
    def add(self,key,value):
        self._data[key]=value
        
    def get(self,key):
        return self._data[key]
    
    def __repr__(self):
        return self._data.__repr__()

data_class_instance = data_class()
data_class_instance.add("A",1)
data_class_instance.add("B","Foo")

print("Value of A:", data_class_instance.get("A"))
print("Value of B:", data_class_instance.get("B"))

Value of A: 1
Value of B: Foo


Since we implemented `__repr__`, I should be able to store the data using `repr`:

In [37]:
with open('file.py',"w") as f: 
    f.write(repr(data_class_instance))

In [38]:
with open('file.py', 'r') as f: 
    data_class_instance_reloaded = eval(f.read())

data_class_instance_reloaded

{'A': 1, 'B': 'Foo'}

But what I get back is not the original object reconstituted, but a dictionary holding the data:

In [39]:
type(data_class_instance_reloaded)

dict

In [40]:
data_class_instance_reloaded.add("C",2)

AttributeError: 'dict' object has no attribute 'add'

In [41]:
data_class_instance_reloaded

{'A': 1, 'B': 'Foo'}

### pickle

Pickle allows me to store the object:

In [42]:
with open('file.pickle',"wb") as f: 
    pickle.dump(data_class_instance,f)

In [43]:
with open('file.pickle', 'rb') as f: 
    data_class_instance_reloaded = pickle.load(f)

data_class_instance_reloaded

{'A': 1, 'B': 'Foo'}

In [44]:
type(data_class_instance_reloaded)

__main__.data_class

In [45]:
data_class_instance_reloaded.add("C",2)

## Storing Multiple Objects into Pickle

Use a dictionary.

In [46]:
data_class_instance_2 = data_class()
data_class_instance_2.add("C",2)
data_class_instance_2.add("D","Bar")

In [47]:
with open('file.pickle',"wb") as f: 
    pickle.dump({"my_class":data_class_instance,
                 "my_class_2":data_class_instance_2},
                f)

In [48]:
with open('file.pickle', 'rb') as f: 
    loaded_data = pickle.load(f)

data_class_instance_reloaded = loaded_data["my_class"]
data_class_instance_reloaded_2 = loaded_data["my_class_2"]

## Pickling Data

In [49]:
import numpy as np
M = np.random.random((1000,1000))

In [50]:
with open('M.pickle',"wb") as f: 
    pickle.dump(M, f)

In [51]:
np.save("M.npy",M)

In [52]:
!ls -lh

total 31640
-rw-r--r--@ 1 afarbin  staff    31K Oct  3 13:48 Lecture.8.a.ipynb
-rw-r--r--@ 1 afarbin  staff   128K Aug 31 11:21 Lecture.8.b.ipynb
-rw-r--r--  1 afarbin  staff   7.6M Oct  3 13:49 M.npy
-rw-r--r--  1 afarbin  staff   7.6M Oct  3 13:49 M.pickle
-rw-r--r--@ 1 afarbin  staff   2.8K Aug 31 11:21 Quiz.2.ipynb
-rw-r--r--@ 1 afarbin  staff   2.4K Aug 31 11:21 Scores.csv
-rw-r--r--  1 afarbin  staff    20B Oct  3 13:35 file.json
-rw-r--r--  1 afarbin  staff   132B Oct  3 13:47 file.pickle
-rw-r--r--  1 afarbin  staff    20B Oct  3 13:44 file.py
-rw-r--r--  1 afarbin  staff    12B Oct  3 13:32 file.yaml


In [53]:
!ls -l

total 31648
-rw-r--r--@ 1 afarbin  staff    33150 Oct  3 13:50 Lecture.8.a.ipynb
-rw-r--r--@ 1 afarbin  staff   131537 Aug 31 11:21 Lecture.8.b.ipynb
-rw-r--r--  1 afarbin  staff  8000128 Oct  3 13:49 M.npy
-rw-r--r--  1 afarbin  staff  8000163 Oct  3 13:49 M.pickle
-rw-r--r--@ 1 afarbin  staff     2896 Aug 31 11:21 Quiz.2.ipynb
-rw-r--r--@ 1 afarbin  staff     2428 Aug 31 11:21 Scores.csv
-rw-r--r--  1 afarbin  staff       20 Oct  3 13:35 file.json
-rw-r--r--  1 afarbin  staff      132 Oct  3 13:47 file.pickle
-rw-r--r--  1 afarbin  staff       20 Oct  3 13:44 file.py
-rw-r--r--  1 afarbin  staff       12 Oct  3 13:32 file.yaml


In [54]:
M_list=M.tolist()

In [55]:
with open('M_list.pickle',"wb") as f: 
    pickle.dump(M, f)

In [56]:
!ls -lh

total 47280
-rw-r--r--@ 1 afarbin  staff    32K Oct  3 13:50 Lecture.8.a.ipynb
-rw-r--r--@ 1 afarbin  staff   128K Aug 31 11:21 Lecture.8.b.ipynb
-rw-r--r--  1 afarbin  staff   7.6M Oct  3 13:49 M.npy
-rw-r--r--  1 afarbin  staff   7.6M Oct  3 13:49 M.pickle
-rw-r--r--  1 afarbin  staff   7.6M Oct  3 13:50 M_list.pickle
-rw-r--r--@ 1 afarbin  staff   2.8K Aug 31 11:21 Quiz.2.ipynb
-rw-r--r--@ 1 afarbin  staff   2.4K Aug 31 11:21 Scores.csv
-rw-r--r--  1 afarbin  staff    20B Oct  3 13:35 file.json
-rw-r--r--  1 afarbin  staff   132B Oct  3 13:47 file.pickle
-rw-r--r--  1 afarbin  staff    20B Oct  3 13:44 file.py
-rw-r--r--  1 afarbin  staff    12B Oct  3 13:32 file.yaml


In [57]:
!ls -l

total 47280
-rw-r--r--@ 1 afarbin  staff    33150 Oct  3 13:50 Lecture.8.a.ipynb
-rw-r--r--@ 1 afarbin  staff   131537 Aug 31 11:21 Lecture.8.b.ipynb
-rw-r--r--  1 afarbin  staff  8000128 Oct  3 13:49 M.npy
-rw-r--r--  1 afarbin  staff  8000163 Oct  3 13:49 M.pickle
-rw-r--r--  1 afarbin  staff  8000163 Oct  3 13:50 M_list.pickle
-rw-r--r--@ 1 afarbin  staff     2896 Aug 31 11:21 Quiz.2.ipynb
-rw-r--r--@ 1 afarbin  staff     2428 Aug 31 11:21 Scores.csv
-rw-r--r--  1 afarbin  staff       20 Oct  3 13:35 file.json
-rw-r--r--  1 afarbin  staff      132 Oct  3 13:47 file.pickle
-rw-r--r--  1 afarbin  staff       20 Oct  3 13:44 file.py
-rw-r--r--  1 afarbin  staff       12 Oct  3 13:32 file.yaml


In [ ]:
!rm *.pickle *.yaml *.json file.py